In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
262,AFG,Asia,Afghanistan,2020-09-18,38872.0,0.0,42.857,1436.0,0.0,2.286,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83,0.498
456,ALB,Europe,Albania,2020-09-18,11948.0,132.0,155.429,348.0,5.0,3.429,...,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57,0.785
719,DZA,Africa,Algeria,2020-09-18,49194.0,228.0,243.714,1654.0,9.0,9.000,...,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88,0.754


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-17,France,412165.0,0.0,31101.0,0.0,0.0,0.0
0,2020-09-17,China,90270.0,0.0,4741.0,0.0,0.0,0.0
0,2020-09-17,Italy,292549.0,0.0,35672.0,0.0,0.0,0.0
0,2020-09-17,Spain,622993.0,0.0,30388.0,0.0,0.0,0.0
0,2020-09-17,United States,6714756.0,0.0,198155.0,0.0,0.0,0.0
0,2020-09-17,World,30382871.0,0.0,949656.0,0.0,0.0,0.0
0,2020-09-17,United Kingdom,380592.0,0.0,41737.0,0.0,0.0,0.0
0,2020-09-17,Germany,267485.0,0.0,9381.0,0.0,0.0,0.0
0,2020-09-17,Iran,413488.0,0.0,23838.0,0.0,0.0,0.0
0,2020-09-17,Turkey,297734.0,0.0,7303.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-18,France,412165.0,415481.0,31101.0,31095.0,-3316.0,6.0
0,2020-09-18,China,90270.0,90294.0,4741.0,4736.0,-24.0,5.0
0,2020-09-18,Italy,292549.0,293025.0,35672.0,35658.0,-476.0,14.0
0,2020-09-17,Spain,622993.0,625651.0,30388.0,30405.0,-2658.0,-17.0
0,2020-09-18,United States,6714756.0,6674458.0,198155.0,197633.0,40298.0,522.0
0,2020-09-18,World,30382871.0,30214496.0,949656.0,946665.0,168375.0,2991.0
0,2020-09-18,United Kingdom,380592.0,381614.0,41737.0,41705.0,-1022.0,32.0
0,2020-09-18,Germany,267485.0,267773.0,9381.0,9378.0,-288.0,3.0
0,2020-09-18,Iran,413488.0,413149.0,23838.0,23808.0,339.0,30.0
0,2020-09-18,Turkey,297734.0,298039.0,7303.0,7315.0,-305.0,-12.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")